In [ ]:
!pip install --upgrade transformers
!pip install simpletransformers

     |████████████████████████████████| 1.5MB 8.8MB/s 
     |████████████████████████████████| 890kB 41.0MB/s 
     |████████████████████████████████| 2.9MB 50.5MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893261 sha256=db0d4de0b83147f7bb63b87875e02c0d8ec2691a36aadaf86f0311ea91d87aed
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 204kB 8.1MB/s 
     |████████████████████████████████| 51kB 8.6MB/s 
     |████████████████████████████████| 7.4MB 18.0MB/s 
     |████████████████████████████████| 317kB 55.9MB/s 
     |████████████████████████████████| 1.8MB 57.2MB/s 
     |████████████████████████████████| 1.1MB 54.5MB/s 
     |████████████████████████████████| 71kB 10.3MB/s 
     |████████████████████████████████| 163kB 57.6MB/s 
     |████████████████████████████████| 4.5MB 47.9MB/s 
     |█████████████████████████████

In [ ]:
# importing important libraries
# checking if the libraries are all good,or is missing thing!
# used try/except to handle it!

try:
  # !pip install --upgrade transformers
  # !pip install simpletransformers
  from sklearn.metrics import f1_score
  from sklearn.model_selection import train_test_split
  from simpletransformers.classification import ClassificationModel, ClassificationArgs
  import pandas as pd
  import numpy as np
  import seaborn as sns
  sns.set_theme(style="darkgrid")

except Exception as e:
  print('Package Error! \n\n {}'.format(e))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#EDA
# calling the train/test data, just to make everything easy!
train = pd.read_csv('/content/drive/MyDrive/hahakathon/train.csv')
test  = pd.read_csv('/content/drive/MyDrive/hahakathon/public_dev.csv')
ssubmit=pd.read_csv('/content/drive/MyDrive/hahakathon/submit.csv')

train.head()

,id,text,is_humor,humor_rating,humor_controversy,offense_rating
0,1,TENNESSEE: We're the best state. Nobody even c...,1,2.42,1.0,0.2
1,2,A man inserted an advertisement in the classif...,1,2.50,1.0,1.1
2,3,How many men does it take to open a can of bee...,1,1.95,0.0,2.4
3,4,Told my mom I hit 1200 Twitter followers. She ...,1,2.11,1.0,0.0
4,5,Roses are dead. Love is fake. Weddings are bas...,1,2.78,0.0,0.1


In [ ]:
train.drop(['is_humor','offense_rating','humor_controversy'], axis=1, inplace=True)

In [ ]:
train.head()

,id,text,humor_rating
0,1,TENNESSEE: We're the best state. Nobody even c...,2.42
1,2,A man inserted an advertisement in the classif...,2.50
2,3,How many men does it take to open a can of bee...,1.95
3,4,Told my mom I hit 1200 Twitter followers. She ...,2.11
4,5,Roses are dead. Love is fake. Weddings are bas...,2.78


In [ ]:
train['humor_rating'].fillna(0, inplace = True) 

In [ ]:
train.isnull().sum()

id              0
text            0
humor_rating    0
dtype: int64

In [ ]:
# init the train columns!
train.columns = ['id','text', 'labels']
print(train.columns)

Index(['id', 'text', 'labels'], dtype='object')


In [ ]:
# declaring the train_df  and valid_df values!
train_df, valid_df = train_test_split(train, test_size=0.01, random_state=10)

In [ ]:

model_args = ClassificationArgs(num_train_epochs=1, learning_rate=1e-5, overwrite_output_dir=True,regression=True, manual_seed=17, silent=True)

model = ClassificationModel(model_type='roberta', model_name='roberta-large' ,use_cuda=True, num_labels=1, args=model_args)

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.o

In [ ]:
# training the model, with the train set!
model.train_model(train_df)

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:136: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:216: UserWarning: Please also save or load the state of the optimizer when saving or loading the scheduler.
  warnings.warn(SAVE_STATE_WARNING, UserWarning)


(990, 0.5913497425352383)

In [ ]:
# extracting the results,
# and then starting the presictions with the test set!!
result, model_outputs, wrong_predictions = model.eval_model(valid_df)
print(f'Results: {result}')

Results: {'eval_loss': 0.3018875602632761}


In [ ]:
#predicting using the test set!
test_predictions, raw_outputs = model.predict(test['text'])

In [ ]:
# saving the result to .csv path file,
# to submit it on codalab!!

id = test['id']
is_humor=ssubmit['is_humor']
humor_rating = test_predictions
submit = pd.DataFrame({'id': id,'is_humor':is_humor ,'humor_rating':humor_rating})
submit.to_csv('submita1b.csv', index=False)

In [ ]:
import zipfile
#declaring the submition,
# and writing to zip file!
submition = zipfile.ZipFile("submita1b.zip", mode='w', compression=zipfile.ZIP_DEFLATED)
submition.write("submita1b.csv")
 
submition.close()